In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os
import numpy as np
from datetime import datetime
from collections  import Counter
from nltk import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
#from wordcloud import WordCloud ,STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
print(os.listdir("."))
from gensim.models import word2vec
from sklearn.manifold import TSNE
from nltk import pos_tag
from nltk.help import upenn_tagset
import gensim
import matplotlib.colors as mcolors
from nltk import jaccard_distance
from nltk import ngrams
#import textstat
plt.style.use('ggplot')

['.ipynb_checkpoints', '80-cereals.zip', 'cereal.csv', 'Lucky_Charms.ipynb', 'Untitled.ipynb']


Using TensorFlow backend.


In [ ]:
job_title=pd.read_csv(csvfiles[0])
sample_job=pd.read_csv(csvfiles[1])
kaggle_data=pd.read_csv(csvfiles[2])

In [6]:
bulletins=os.listdir("LA-Good-City/CityofLA/Job Bulletins/")
additional=os.listdir("LA-Good-City/CityofLA//Additional data/")

In [7]:
csvfiles=[]
for file in additional:
    if file.endswith('.csv'):
        print(file)
        csvfiles.append("LA-Good-City/CityofLA/Additional data/"+file)
        

job_titles.csv
kaggle_data_dictionary.csv
sample job class export template.csv


In [10]:
job_title=pd.read_csv(csvfiles[0])
sample_job=pd.read_csv(csvfiles[1])
kaggle_data=pd.read_csv(csvfiles[2])

In [11]:
job_title.head()

,311 DIRECTOR
0,ACCOUNTANT
1,ACCOUNTING CLERK
2,ACCOUNTING RECORDS SUPERVISOR
3,ADMINISTRATIVE ANALYST
4,ADMINISTRATIVE CLERK


In [14]:
print("The are %d rows and %d cols in sample_job file" %(sample_job.shape))


The are 25 rows and 7 cols in sample_job file


In [15]:
kaggle_data.head()

,FILE_NAME,JOB_CLASS_TITLE,JOB_CLASS_NO,REQUIREMENT_SET_ID,REQUIREMENT_SUBSET_ID,JOB_DUTIES,EDUCATION_YEARS,SCHOOL_TYPE,EDUCATION_MAJOR,EXPERIENCE_LENGTH,...,COURSE_LENGTH,COURSE_SUBJECT,MISC_COURSE_DETAILS,DRIVERS_LICENSE_REQ,DRIV_LIC_TYPE,ADDTL_LIC,EXAM_TYPE,ENTRY_SALARY_GEN,ENTRY_SALARY_DWP,OPEN_DATE
0,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,1,A,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,COMPUTER SCIENCE|INFORMATION SYSTEMS|GEOGRAPHI...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
1,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,A,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
2,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,B,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
3,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,2,C,"A Systems Analyst analyzes procedures, methods...",4.0,COLLEGE OR UNIVERSITY,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17
4,SYSTEMS ANALYST 1596 102717.txt,Systems Analyst,1596,3,A,"A Systems Analyst analyzes procedures, methods...",NaN,NaN,NaN,2.0,...,"3M, 4Q",INFORMATION SYSTEMS|SYSTEMS ANALYSIS|closely r...,At least three of the courses must be from the...,M,NaN,NaN,OPEN_INT_PROM,68611-100307,70908-88092,10/27/17


In [16]:
print("The are %d rows and %d cols in kaggle_data file" %(kaggle_data.shape))

The are 5 rows and 25 cols in kaggle_data file


In [17]:
print("There are %d text files in bulletin directory" %len(bulletins))

There are 683 text files in bulletin directory


In [20]:
def get_headings(bulletin):       
    
    """"function to get the headings from text file
        takes a single argument
        1.takes single argument list of bulletin files"""
    
    with open("LA-Good-City/CityofLA/Job Bulletins/"+bulletins[bulletin]) as f:    ##reading text files 
        data=f.read().replace('\t','').split('\n')
        data=[head for head in data if head.isupper()]
        return data
        
def clean_text(bulletin):      
    
    
    """function to do basic data cleaning
        takes a single argument
        1.takes single argument list of bulletin files"""
                                            
    
    with open("LA-Good-City/CityofLA/Job Bulletins/"+bulletins[bulletin]) as f:
        data=f.read().replace('\t','').replace('\n','')
        return data

In [21]:
get_headings(1)

['ACCOUNTANT',
 'ANNUAL SALARY',
 'NOTES:',
 'DUTIES',
 'REQUIREMENT/MINIMUM QUALIFICATION',
 'PROCESS NOTES',
 'WHERE TO APPLY',
 'NOTE:',
 'APPLICATION DEADLINE',
 'SELECTION PROCESS',
 'NOTES:',
 'NOTICE:',
 'THIS EXAMINATION IS TO BE GIVEN ONLY',
 'ON AN OPEN COMPETITIVE BASIS',
 'AN EQUAL EMPLOYMENT OPPORTUNITY EMPLOYER']

In [22]:
get_headings(2)

['ACCOUNTING CLERK',
 'ANNUAL SALARY',
 'NOTE:',
 'DUTIES',
 'REQUIREMENT/MINIMUM QUALIFICATION',
 'PROCESS NOTES',
 'WHERE TO APPLY',
 'NOTE:',
 'APPLICATION DEADLINE',
 'SELECTION PROCESS',
 'NOTES:',
 'NOTICE:',
 'THIS EXAMINATION IS TO BE GIVEN BOTH ON AN',
 'INTERDEPARTMENTAL PROMOTIONAL AND OPEN COMPETITIVE BASIS',
 'AN EQUAL EMPLOYMENT OPPORTUNITY EMPLOYER']

In [24]:



def to_dataframe(num,df):
    """"function to extract features from job bulletin text files and convert to
    pandas dataframe.
    function take two arguments 
                        1.the number of files to be read
                        2.dataframe object                                      """
    

    
    opendate=re.compile(r'(Open [D,d]ate:)(\s+)(\d\d-\d\d-\d\d)')       #match open date
    
    salary=re.compile(r'\$(\d+,\d+)((\s(to|and)\s)(\$\d+,\d+))?')       #match salary
    
    requirements=re.compile(r'(REQUIREMENTS?/\s?MINIMUM QUALIFICATIONS?)(.*)(PROCESS NOTE)')      #match requirements
    
    for no in range(0,num):
        with open("LA-Good-City/CityofLA/Job Bulletins/"+bulletins[no],encoding="ISO-8859-1") as f:         #reading files 
                try:
                    file=f.read().replace('\t','')
                    data=file.replace('\n','')
                    headings=[heading for heading in file.split('\n') if heading.isupper()]             ##getting heading from job bulletin

                    sal=re.search(salary,data)
                    date=datetime.strptime(re.search(opendate,data).group(3),'%m-%d-%y')
                    try:
                        req=re.search(requirements,data).group(2)
                    except Exception as e:
                        req=re.search('(.*)NOTES?',re.findall(r'(REQUIREMENTS?)(.*)(NOTES?)',
                                                              data)[0][1][:1200]).group(1)
                    
                    duties=re.search(r'(DUTIES)(.*)(REQ[A-Z])',data).group(2)
                    try:
                        enddate=re.search(
                                r'(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)\s(\d{1,2},\s\d{4})'
                                ,data).group()
                    except Exception as e:
                        enddate=np.nan
                    
                    selection= [z[0] for z in re.findall('([A-Z][a-z]+)((\s\.\s)+)',data)]     ##match selection criteria
                    
                    df=df.append({'File Name':bulletins[no],'Position':headings[0].lower(),'salary_start':sal.group(1),
                               'salary_end':sal.group(5),"opendate":date,"requirements":req,'duties':duties,
                                'deadline':enddate,'selection':selection},ignore_index=True)
                    
                    
                    reg=re.compile(r'(One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten|one|two|three|four)\s(years?)\s(of\sfull(-|\s)time)')
                    df['EXPERIENCE_LENGTH']=df['requirements'].apply(lambda x :  re.search(reg,x).group(1) if re.search(reg,x) is not None  else np.nan)
                    df['FULL_TIME_PART_TIME']=df['EXPERIENCE_LENGTH'].apply(lambda x:  'FULL_TIME' if x is not np.nan else np.nan )
                    
                    reg=re.compile(r'(One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten|one|two|three|four)(\s|-)(years?)\s(college)')
                    df['EDUCATION_YEARS']=df['requirements'].apply(lambda x :  re.search(reg,x).group(1) if re.search(reg,x) is not None  else np.nan)
                    df['SCHOOL_TYPE']=df['EDUCATION_YEARS'].apply(lambda x : 'College or University' if x is not np.nan else np.nan)
                    
                except Exception as e:
                    print('umatched sequence')
                    
                    
                
                
        
           
    return df

            
            
            
            

In [25]:
df=pd.DataFrame(columns=['File Name','Position',
                         'salary_start','salary_end',
                         'opendate','requirements',
                         'duties','deadline'])
df=to_dataframe(len(bulletins),df)
df.to_csv('job class output.csv')

umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence
umatched sequence


In [ ]:
#To convert some columns from a data frame to a list of dicts, we call df.to_dict( orient = 'records' ) (thanks to José P. González-Brenes for the tip):

cols_to_retain = [ 'a', 'list', 'of', 'categorical', 'column', 'names' ]
cat_dict = df[ cols_to_retain ].to_dict( orient = 'records' )
#If you have a few categorical columns, you can list them as above. In the Analytics Edge competition, there are about 100 categorical columns, so in this case it’s easier to drop columns which are not categorical:

cols_to_drop = [ 'UserID, 'YOB', 'votes', 'Happy' ]
cat_dict = df.drop( cols_to_drop, axis = 1 ).to_dict( orient = 'records' )